In [2]:
from tqdm import tqdm
import torch
import pickle
import os
from torch.utils.data import DataLoader
import sys
import trimesh
import numpy as np
from trimesh import voxel

from Encoder.Models.EncoderModels import TextEncoder
from Encoder.dataEmbedding.dataEmbeddingLoader import GenerateDataLoader,check_dataset,collate_embedding
from Encoder.dataEmbedding.dataEmbedding import Read_Load_BuildBatch

from config import cfg
import random

sys.path.append('./WGAN/')


from WGAN.Script.scripts import sample_z,vizualizer_stanford_data

from WGAN.Models.Generator import Generator32_Small
from WGAN.DataLoader.GanDataGen import GANDataGenerator

from WGAN.DataLoader.GANloader import GANLoader
from WGAN.Solvers.SolverGAN import SolverGAN


def singleRunGenerator(prompt):
    os.chdir(os.path.join(os.getcwd(), "Encoder"))
    
    
    stanData=Read_Load_BuildBatch(cfg.EMBEDDING_BATCH_SIZE)



    model=TextEncoder(len(stanData.dict_word2idx))
    model.load_state_dict(torch.load('./../SavedModels/text_encoder.pth',map_location=torch.device('cpu')))
    model=model.to(cfg.DEVICE)
    model.eval()
    

    new_dataset=GenerateDataLoader([('None',0,0,prompt)],stanData.dict_word2idx,'test')
    loader = DataLoader(new_dataset, batch_size=1,collate_fn=collate_embedding)

    os.chdir("..")

    data_iterator = iter(loader)
    single_element = next(data_iterator)

    texts = single_element[3].to(cfg.DEVICE)

    text_embedding = model(texts)

    
    text_embedding=torch.cat((text_embedding,sample_z()),1).squeeze(0)

    os.chdir(os.path.join(os.getcwd(), "WGAN"))

    generator=Generator32_Small()
    generator.load_state_dict(torch.load(os.path.join('./../SavedModels',"generator_model6.pth"),map_location=torch.device('cpu')))
    generator.eval()
    
    os.chdir("..")

    gen=generator(text_embedding.unsqueeze(0))

    gen=gen['sigmoid_output'].squeeze(0).detach()
    vizualizer_stanford_data(gen,'fake')

In [3]:
import torch

# Create a binary tensor of size 32x32x32 with values 0 and 1
tensor_size = (32, 32, 32)
binary_tensor = torch.randint(2, size=tensor_size, dtype=torch.int)

# Find the coordinates of non-zero elements
coordinates = torch.nonzero(binary_tensor)

# Print the coordinates
print(coordinates)
coordinate_list = list(map(tuple, coordinates.tolist()))
print(coordinate_list)

tensor([[ 0,  0,  0],
        [ 0,  0,  2],
        [ 0,  0,  3],
        ...,
        [31, 31, 26],
        [31, 31, 27],
        [31, 31, 31]])
[(0, 0, 0), (0, 0, 2), (0, 0, 3), (0, 0, 6), (0, 0, 7), (0, 0, 8), (0, 0, 9), (0, 0, 10), (0, 0, 11), (0, 0, 13), (0, 0, 14), (0, 0, 15), (0, 0, 17), (0, 0, 18), (0, 0, 19), (0, 0, 21), (0, 0, 22), (0, 0, 27), (0, 0, 28), (0, 0, 31), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 5), (0, 1, 6), (0, 1, 8), (0, 1, 10), (0, 1, 15), (0, 1, 16), (0, 1, 18), (0, 1, 20), (0, 1, 25), (0, 1, 26), (0, 1, 29), (0, 2, 0), (0, 2, 4), (0, 2, 10), (0, 2, 14), (0, 2, 15), (0, 2, 17), (0, 2, 19), (0, 2, 21), (0, 2, 23), (0, 2, 25), (0, 2, 26), (0, 2, 28), (0, 2, 30), (0, 3, 0), (0, 3, 1), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 10), (0, 3, 11), (0, 3, 12), (0, 3, 13), (0, 3, 15), (0, 3, 19), (0, 3, 21), (0, 3, 22), (0, 3, 23), (0, 3, 25), (0, 3, 28), (0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 4, 5), (0, 4, 6), (0, 4, 8), (0, 4, 10), (0, 4, 11), (0, 4, 12), (0, 

In [5]:
singleRunGenerator("chair")

chair
